In [1]:
import os

In [2]:
%pwd

'c:\\Python\\Chicken_Disease_Classification\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Python\\Chicken_Disease_Classification\\Chicken-Disease-Classification'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data : Path
    all_params: dict
    params_image_size: list
    params_batch_size :int



In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [20]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= "artifacts/training/model.keras",
            training_data = "artifacts/data_ingestion/Chicken_fecal_images",
            all_params=self.params,
            params_batch_size=self.params.BATCH_SIZE,
            params_image_size=self.params.IMAGE_SIZE
        )
        return eval_config

In [21]:
from urllib.parse import urlparse

In [27]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:2],  # cần lấy cả (height, width)
            batch_size=self.config.params_batch_size,
            interpolation='bilinear',
            class_mode='categorical',
            subset='validation',
            seed=42
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            **dataflow_kwargs
        )

        return valid_generator

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        valid_generator = self._valid_generator()
        self.score = self.model.evaluate(valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [28]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2025-05-07 13:41:32,346: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-07 13:41:32,351: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-07 13:41:32,354: INFO: common: created directory at: artifacts]
Found 19 images belonging to 2 classes.


c:\Python\Chicken_Disease_Classification\Chicken-Disease-Classification\chicken\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 816ms/step - accuracy: 0.4616 - loss: 1.5732
[2025-05-07 13:41:40,421: INFO: common: json file saved at: scores.json]
